In [1]:
!nvidia-smi

Tue Mar 26 14:50:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.76                 Driver Version: 551.76         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P0             20W /   95W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install -q git+https://github.com/facebookresearch/segment-anything.git

In [4]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

In [6]:
!pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/e7/45/419aa0b37254f1fd62b45bb63836066c5eb81e37d70940e0491e95167eed/torchvision-0.17.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.2 MB 1.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.1/1.2 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 8.2 MB/s eta 0:00:00


In [1]:
import os
import cv2
import numpy as np
import supervision as sv
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [4]:
HOME = os.getcwd()
print("HOME:", HOME)

HOME: C:\Users\User\Downloads


In [15]:
import requests
import os

def download_file(url, save_path):
    # Send a GET request to the URL
    response = requests.get(url, stream=True)
    
    # Ensure the request was successful
    if response.status_code == 200:
        # Make sure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        # Open the file in binary write mode and save the content to the file
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"File downloaded successfully: {save_path}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

# Define the URL
url = 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'

# Now include the filename in the save_path
save_path = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")

# Download the file
download_file(url, save_path)


File downloaded successfully: C:\Users\User\Downloads\weights\sam_vit_h_4b8939.pth


In [3]:
CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

C:\Users\User\Downloads\weights\sam_vit_h_4b8939.pth ; exist: True


In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
print(DEVICE)

cuda:0


In [5]:
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [6]:
mask_predictor = SamPredictor(sam)

In [7]:
default_box = {'x': 132, 'y': 151, 'width': 300, 'height': 300, 'label': ''}

box = default_box
box = np.array([
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
])

In [8]:
source_dir = os.path.join(HOME, 'Original/train/')
os.makedirs(source_dir, exist_ok=True)

In [11]:
import os
import cv2
import numpy as np

source_dir = os.path.join(HOME, 'Original/train/')
target_root_dir = os.path.join(HOME, 'Original_segmentation/train/')

os.makedirs(target_root_dir, exist_ok=True)

for folder_name in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder_name)

    target_folder_path = os.path.join(target_root_dir, folder_name)
    os.makedirs(target_folder_path, exist_ok=True)

    for image_filename in os.listdir(folder_path):
        IMAGE_PATH = os.path.join(folder_path, image_filename)
        image_bgr = cv2.imread(IMAGE_PATH)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        mask_predictor.set_image(image_rgb)

        masks, scores, logits = mask_predictor.predict(
            box=box,
            multimask_output=True
        )

        mask = masks[np.argmax(scores)]
        mask = mask.astype(bool)
        masked_image = np.zeros_like(image_rgb)

        for c in range(3):
            masked_image[:, :, c] = image_rgb[:, :, c] * mask

        y_indices, x_indices = np.where(mask)
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()

        cropped_image = masked_image[y_min:y_max, x_min:x_max]

        cropped_image_path = os.path.join(target_folder_path, image_filename + "_segment.png")
        cv2.imwrite(cropped_image_path, cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))


In [ ]:
box_annotator = sv.BoxAnnotator(color=sv.Color.red())
mask_annotator = sv.MaskAnnotator(color=sv.Color.red(), color_lookup=sv.ColorLookup.INDEX)

detections = sv.Detections(
    xyxy=sv.mask_to_xyxy(masks=masks),
    mask=masks
)
detections = detections[detections.area == np.max(detections.area)]

source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections, skip_label=True)
segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[source_image, segmented_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)

In [ ]:
# helper function that loads an image before adding it to the widget

import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [ ]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

widget = BBoxWidget()
widget.image = encode_image("/content/DWT_GrayScale_db1_1%_100.bmp")
widget

In [ ]:
widget.bboxes

In [12]:
!pip install accelerate -U datasets transformers[torch] evaluate

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/0a/fd/280f4385e76f3c1890efc15fa93f7206134fefad6351397e1bfab6d0d0de/transformers-4.39.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? et

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.2.0 which is incompatible.


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
"""import os
import shutil
import pandas as pd

csv_path = "/content/Class Labels of Dataset 2.csv"
labels_df = pd.read_csv(csv_path)

source_dir = '/content/'
target_base_dir = '/content/train/'

for _, row in labels_df.iterrows():
    class_dir = os.path.join(target_base_dir, str(row['class']))
    os.makedirs(class_dir, exist_ok=True)

    source_path = os.path.join(source_dir, f"DWT_RGB_db1_1%_{str(row['image ID']).zfill(3)}_segment.png")
    target_path = os.path.join(class_dir, f"DWT_RGB_db1_1%_{str(row['image ID']).zfill(3)}_segment.png")

    shutil.move(source_path, target_path)"""

Files have been reorganized.


In [5]:
import os
from datasets import load_dataset
data_dir = os.path.join(HOME, 'Original_segmentation/train/')
dataset = load_dataset("imagefolder", data_dir=data_dir)
wbc = dataset
print(wbc)

Resolving data files:   0%|          | 0/16633 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 16633
    })
})


In [11]:
print(wbc['train'][0])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=263x229 at 0x1E2059E6C90>, 'label': 4}


In [14]:
from collections import defaultdict

# Initialize a dictionary to count the number of photos per label
label_counts = defaultdict(int)

# Iterate through the 'train' split of your dataset
for item in wbc['train']:
    # Increment the count for the item's label
    label_counts[item['label']] += 1

# Print the count of photos for each label
for label, count in label_counts.items():
    print(f"Label {label}: {count} photos")


Label 0: 301 photos
Label 1: 1066 photos
Label 2: 3609 photos
Label 3: 795 photos
Label 4: 10862 photos


In [15]:
label_names = ['Basophil','Eosinophil','Lymphocyte','Monocyte','Neutrophil']
id2label = {id: label for id, label in enumerate(label_names)}
label2id = {label: id for id, label in id2label.items()}

In [25]:
print(id2label[4])

Neutrophil


In [13]:
wbc.push_to_hub("Boom4869/Original_WBC", private=True)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/8317 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/84 [00:00<?, ?ba/s]

Map:   0%|          | 0/8316 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/84 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Boom4869/Original_WBC/commit/aaa5714135472764bae6e6ef8fcef13218bde6a4', commit_message='Upload dataset', commit_description='', oid='aaa5714135472764bae6e6ef8fcef13218bde6a4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
train_test_split = wbc['train'].train_test_split(test_size=0.3)
wbc['train'] = train_test_split['train']
wbc['test'] = train_test_split['test']

validation_test_split = wbc['test'].train_test_split(test_size=0.5)
wbc['val'] = validation_test_split['train']
wbc['test'] = validation_test_split['test']


NameError: name 'wbc' is not defined

In [ ]:
label_names = ['Neutrophil', 'Lymphocyte', 'Monocyte', 'Eosinophil', 'Basophil']
id2label = {id: label for id, label in enumerate(label_names)}
label2id = {label: id for id, label in id2label.items()}

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
wbc = wbc.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels= 5,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_wbc_model_Original",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=wbc["train"],
    eval_dataset=wbc["val"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.884197,0.150000
1,No log,1.628194,0.300000
2,No log,1.426221,0.400000
4,No log,1.191200,0.500000
5,No log,1.168453,0.650000
6,No log,1.092500,0.550000
8,1.218700,1.043836,0.750000
9,1.218700,1.033991,0.650000
10,1.218700,0.866847,0.800000
12,1.218700,0.954729,0.800000


Checkpoint destination directory my_awesome_wbc_model/checkpoint-1 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_wbc_model/checkpoint-2 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_wbc_model/checkpoint-3 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_wbc_model/checkpoint-5 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_wbc_model/checkpoint-6 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_wbc_model/checkpoint-7 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_awesome_wbc_model/checkpoint-8 already e

TrainOutput(global_step=16, training_loss=0.9962332993745804, metrics={'train_runtime': 116.5509, 'train_samples_per_second': 10.982, 'train_steps_per_second': 0.137, 'total_flos': 7.935393098052403e+16, 'train_loss': 0.9962332993745804, 'epoch': 12.8})

In [ ]:
test_results = trainer.evaluate(wbc["test"])
print(test_results)